In [23]:
import csv, glob, json, os, re, shutil, subprocess, sys, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

def download_file(url, filename):
    if os.path.exists(filename):
        sys.stdout.write('%s already downloaded\n' % filename)
    else:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        sys.stdout.write('Downloading %s to %s\n' % (url, filename))
        data = urllib2.urlopen(url).read()
        open(filename + '.tmp', "wb").write(data)
        os.rename(filename + '.tmp', filename)
        sys.stdout.write('Done, wrote %d bytes to %s\n' % (len(data), filename))
        
def unzip_file(filename):
    exdir = os.path.splitext(filename)[0]
    if os.path.exists(exdir):
        sys.stdout.write('%s already unzipped\n' % (filename))
    else:
        tmpdir = exdir + '.tmp'
        shutil.rmtree(tmpdir, True)
        sys.stdout.write('Unzipping %s into %s\n' % (filename, tmpdir))
        subprocess.check_call(['unzip', filename, '-d', tmpdir])
        os.rename(tmpdir, exdir)
        print 'Success, created %s' % (exdir)

In [24]:
def sanitize(f):
    return re.sub(r'\W+', '_', f).strip('_')

# print sanitize('    a b c    d')

def remove_spaces(n):
    return n.replace(' ', '')

In [25]:
download_file('http://api.worldbank.org/v2/en/indicator/EG.ELC.ACCS.ZS?downloadformat=csv', 'WorldBank/EG.ELC.ACCS.ZS.zip')
unzip_file('WorldBank/EG.ELC.ACCS.ZS.zip')
files = glob.glob('WorldBank/EG.ELC.ACCS.ZS/API*csv')
assert len(files) == 1
file = files[0]
print 'Data is in %s' % file

WorldBank/EG.ELC.ACCS.ZS.zip already downloaded
WorldBank/EG.ELC.ACCS.ZS.zip already unzipped
Data is in WorldBank/EG.ELC.ACCS.ZS/API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv


In [21]:
!ls WorldBank/EG.ELC.ACCS.ZS
!open WorldBank/EG.ELC.ACCS.ZS 

API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv
Metadata_Country_API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv
Metadata_Indicator_API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv


In [8]:
exec_ipynb('placenames.ipynb')






irena_csv = 'IRENA/IRENA all 2017 05 02.csv'
reader = csv.reader(open(irena_csv))

# Skip 7 rows
for _ in range(0, 7):
    reader.next()
    
# header is 8th row
header = reader.next()

# Find valid data columns
re_yyyy = r'\d\d\d\d'

technology_col = 0
indicator_col = 1
country_col = 2
first_year_col = 3
assert not re.match(re_yyyy, header[first_year_col - 1])
assert re.match(re_yyyy, header[first_year_col])

for i in range(first_year_col + 1, len(header)):
    if not re.match(re_yyyy, header[i]):
        last_year_col = i - 1
        break

print 'Found years %s-%s' % (header[first_year_col], header[last_year_col])

current_technology = None
csv_filename = None
csv_file = None
csv_nrows = 0

def finish_csv():
    global csv_nrows
    if csv_filename:
        csv_file.close()
        os.rename(csv_filename + '.tmp', csv_filename)
        print 'Created %s with %d rows' % (csv_filename, csv_nrows)
    csv_nrows = 0
        

errors = 0

for rec in reader:
    technology = rec[technology_col]
    indicator = rec[indicator_col]
    country = rec[country_col]
    if technology:
        assert indicator
        current_technology = technology
    if indicator:
        finish_csv()
        csv_filename = 'IRENA/%s.%s.csv' % (sanitize(current_technology), sanitize(indicator))
        csv_file = open(csv_filename + '.tmp', 'w')
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([''] + header[first_year_col:last_year_col+1])
    country = canonicalize_country_name(country)
    if not country:
        continue
    if country == 'Error':
        errors += 1
        continue
    csv_writer.writerow([country] + [remove_spaces(x) for x in rec[first_year_col:last_year_col+1]])
    csv_nrows += 1

if errors:
    raise Exception('Errors')
finish_csv()

In [ ]:
!cat IRENA/Solar.Electricity_capacity_MW.csv

In [38]:
!rsync -av IRENA tm1:/usr4/web/data.cmucreatelab.org/www/earthtime

building file list ... done
IRENA/
IRENA/Bagasse.Electricity_capacity_MW.csv
IRENA/Bagasse.Electricity_generation_GWh.csv
IRENA/Bioenergy.Electricity_capacity_MW.csv
IRENA/Bioenergy.Electricity_generation_GWh.csv
IRENA/Biogas.Electricity_capacity_MW.csv
IRENA/Biogas.Electricity_generation_GWh.csv
IRENA/Concentrated_solar_power.Electricity_capacity_MW.csv
IRENA/Concentrated_solar_power.Electricity_generation_GWh.csv
IRENA/Hydro_10_MW.Electricity_capacity_MW.csv
IRENA/Hydro_10_MW.Electricity_generation_GWh.csv
IRENA/Hydro_1_10_MW.Electricity_capacity_MW.csv
IRENA/Hydro_1_10_MW.Electricity_generation_GWh.csv
IRENA/Hydro_1_MW.Electricity_capacity_MW.csv
IRENA/Hydro_1_MW.Electricity_generation_GWh.csv
IRENA/Hydropower.Electricity_capacity_MW.csv
IRENA/Hydropower.Electricity_generation_GWh.csv
IRENA/Liquid_biofuels.Electricity_capacity_MW.csv
IRENA/Liquid_biofuels.Electricity_generation_GWh.csv
IRENA/Marine.Electricity_capacity_MW.csv
IRENA/Marine.Electricity_generation_GWh.csv
IRENA/Mixed_p